In [1]:
from irbasis_util.four_point_ph_view import *
from irbasis_util.internal import *
from irbasis_util.regression import *
from irbasis_util.tensor_regression import *
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#padding on the y axis between label and axis
plt.rcParams['ytick.major.pad']='10'

dpi_out = 400
fig_width = 3.39
golden_mean = (np.sqrt(5)-1.0)/2.0    # Aesthetic ratio
fig_height = fig_width*golden_mean # height in inches
MAX_HEIGHT_INCHES = 8.0
if fig_height > MAX_HEIGHT_INCHES:
    print("WARNING: fig_height too large:" + fig_height + 
          "so will reduce to" + MAX_HEIGHT_INCHES + "inches.")
    fig_height = MAX_HEIGHT_INCHES
#fig, ax = plt.subplots(figsize = (fig_width, fig_height), frameon=True)
small_tick_size = 8
small_label_size = 8

In [3]:
# Implementation of Rohringer et al. () 2012)
def _delta(i, j):
    if i==j:
        return 1
    else:
        return 0

def _F_ph(U, beta, n, np, m):
    nu = (2 * n + 1) * numpy.pi / beta
    nu_p = (2 * np + 1) * numpy.pi / beta
    omega = 2 * m * numpy.pi / beta
    r1 = nu + omega
    r2 = nu_p + omega
    tmp = 1. / (nu * r1 * r2 * nu_p)
    Fuu = (-0.25 * beta * (U**2) * (_delta(n,np) - _delta(m,0)) *
               (1 + 0.25 * (U / nu)**2) * (1 + 0.25 * (U / r2)**2))
    t1 = 0.125 * (U**3) * (nu**2 + r1**2 + r2**2 + nu_p**2) * tmp
    t2 = (3.0 / 16.0) * (U**5) * tmp
    t3 = (beta * 0.25 * (U**2) *
              (1 / (1 + numpy.exp(0.5 * beta * U)))
              * (2 * _delta(nu, -nu_p - m) + _delta(m, 0)) *
              (1 + 0.25 * (U / r1)**2) * (1 + 0.25 * (U / r2)**2))
    t4 = (-beta * 0.25 * (U**2) *
              (1 / (1 + numpy.exp(-0.5 * beta * U)))
              * (2 * _delta(nu, nu_p) + _delta(m, 0)) *
              (1 + 0.25 * (U / nu)**2) * (1 + 0.25 * (U / r2)**2))
    Fud = -U + t1 + t2 + t3 + t4
    return Fuu, Fud

def _G2_conn_ph(U, beta, n, np, m):
    Fuu, Fud = _F_ph(U, beta, n, np, m)
    nu = (2 * n + 1) * numpy.pi / beta
    nu_p = (2 * np + 1) * numpy.pi / beta
    omega = 2 * m * numpy.pi / beta
    hU = 0.5 * U
    leggs_1 = nu * (nu + omega) * nu_p * (nu_p + omega)
    leggs_2 = ((hU**2 + nu**2) * (hU**2 + nu_p**2) *
                   (hU**2 + (nu + omega)**2) * (hU**2 + (nu_p + omega)**2))
    leggs = leggs_1 / leggs_2
    return (leggs * Fuu + leggs * Fud)

In [4]:
def elbow_method(alphas, squared_errors):
    """
    
    Parameters
    ----------
    alphas    regularization parameters in ascending order
    squared_errors  corresponding squared_errors

    Returns index of optimal value of alpha in alphas
    -------

    """
    x = numpy.log(alphas)
    y = numpy.log(squared_errors)
    
    a = (y[-1] - y[0])/(x[-1] - x[0])
    y -= a * (x - x[0]) + y[0]
    
    r = numpy.argmin(y)
    if isinstance(r, numpy.int64):
        return r
    else:
        return r[0]

# Regression with multiple values of alpha
def kruskal_complex_alphas(tensors_A, y, alphas, rank, decomposed_S):
    """
    
    Parameters
    ----------
    A
    y
    alphas  regularization parameters in ascending order

    Returns
    -------
    
     Nw, Nr, freq_dim, linear_dim, tensors_A, y, alpha, D):

    """
    coeffs_alpha = []
    squared_errors_alpha = []
    Nw, Nr, linear_dim = tensors_A[0].shape
    print(Nw, Nr, linear_dim)
    for i, alpha in enumerate(alphas):
        #print(tensors_A)
        #print(y)
        print("alpha ", alpha)
        precond = None
        model = OvercompleteGFModel(Nw, Nr, 2, linear_dim, tensors_A, y, alpha, rank)
        info = optimize_als(model, 1000, tol_rmse = 1e-4, solver='lsqr', verbose = 1)
        coeffs = model.full_tensor_x().numpy()
        coeffs_alpha.append(coeffs)
        e = model.mse().numpy()
        print("alpha = ", alpha, e, " num_ite", len(info['losss']))
        squared_errors_alpha.append(e)
    
    squared_errors_alpha = numpy.array(squared_errors_alpha)
        
    # Choose optimal alpha using Elbow method 
    opt_idx = elbow_method(alphas, squared_errors_alpha)
    
    return coeffs_alpha, squared_errors_alpha, opt_idx

In [5]:
boson_freq = 10
Lambda = 100.0
beta = 5.0
U = 2.0
wmax = Lambda / beta
rank = 10

numpy.random.seed(100)
#noise = 0.0012
# 2% error
noise = 1e-4

In [6]:
#alpha = 1e-15
augmented = True
phb = FourPointPHView(boson_freq, Lambda, beta, 1e-5, augmented)
Nl = phb.Nl
whichl = Nl - 1
print("Nl", Nl)
# build the sampling frequency structure
sp = phb.sampling_points_matsubara(whichl)
#print(S.shape)
n_sp = len(sp)

Nl 19


In [7]:
# prj: [(n_sp, 3, 2, 2, Nl), (n_sp, 3, 2, 2, Nl)]
# S: [(3, 2, 2, Nl), (3, 2, 2, Nl)]
# prj = S * prj
def construct_prj(sp):
    n_sp = len(sp)
    prj = phb.projector_to_matsubara_vec(sp, decomposed_form=True)
    S = phb.normalized_S(decomposed_form=True)
    for i in range(2):
        prj[i] *= S[i][None, :, :, :, :]
        prj[i] = prj[i].reshape((n_sp, 12, Nl))
    return prj
prj = construct_prj(sp)
S_decomposed = phb.normalized_S(decomposed_form=True)

# Build the check frequency structure
n1n2_check = []
niw = 100
niw_hf = 10000
wide_niw_check = numpy.hstack((range(-niw_hf, -niw, 500), range(-niw, niw, 10), range(niw, niw_hf, 500)))
for i, j in product(wide_niw_check, repeat=2):
    n1n2_check.append((i, j))
prj_check = construct_prj(n1n2_check)

numpy.random.seed(200)
Giwn = numpy.array([_G2_conn_ph(U, beta, n1n2[0], n1n2[1], boson_freq) for n1n2 in sp])
print ("adding noise")
noise_iwn = numpy.random.normal(loc=0.0, scale=noise, size=(len(sp)))
Giwn = noise_iwn + Giwn
#print(Giwn.imag)

adding noise


In [ ]:
#numpy.seterr(all='raise')
#alphas = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
alphas = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
#alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 5e-1, 1, 2, 10, 100]
#alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 5e-1, 1, 2, 10, 100]
#print(S_decomposed)
#alphas = [1e-7]
%time coeffs_alpha, se_alpha, opt_alpha_idx = kruskal_complex_alphas(prj, Giwn, alphas, rank, S_decomposed)

2736 12 19
alpha  1
Number of iterations in LSQR =  147
Number of iterations in LSQR =  6
Number of iterations in LSQR =  3
epoch =  0  loss =  8.43414027070638e-06  rmse =  0.0028274807167256396  coeff =  1.0
norm of x  0 (0.030331145481712722+0j)
norm of x  1 (0.016806357529729754+0j)
norm of x  2 (0.0001448583318394213+0j)
Number of iterations in LSQR =  2
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LSQR =  1
Number of iterations in LS

The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of i

The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of iterations in LSQR =  0
The exact solution is  x = 0                              
Number of i

Number of iterations in LSQR =  36
Number of iterations in LSQR =  38
Number of iterations in LSQR =  102
Number of iterations in LSQR =  35
Number of iterations in LSQR =  36
Number of iterations in LSQR =  94
Number of iterations in LSQR =  34
Number of iterations in LSQR =  34
Number of iterations in LSQR =  88
Number of iterations in LSQR =  32
Number of iterations in LSQR =  32
Number of iterations in LSQR =  76
Number of iterations in LSQR =  30
Number of iterations in LSQR =  31
Number of iterations in LSQR =  67
Number of iterations in LSQR =  28
Number of iterations in LSQR =  30
Number of iterations in LSQR =  65
Number of iterations in LSQR =  29
Number of iterations in LSQR =  30
Number of iterations in LSQR =  61
Number of iterations in LSQR =  28
Number of iterations in LSQR =  30
Number of iterations in LSQR =  61
Number of iterations in LSQR =  28
Number of iterations in LSQR =  30
Number of iterations in LSQR =  60
Number of iterations in LSQR =  28
Number of iteration

Number of iterations in LSQR =  174
Number of iterations in LSQR =  62
Number of iterations in LSQR =  81
Number of iterations in LSQR =  174
Number of iterations in LSQR =  63
Number of iterations in LSQR =  82
Number of iterations in LSQR =  164
Number of iterations in LSQR =  59
Number of iterations in LSQR =  83
Number of iterations in LSQR =  167
Number of iterations in LSQR =  59
Number of iterations in LSQR =  82
Number of iterations in LSQR =  163
Number of iterations in LSQR =  59
Number of iterations in LSQR =  81
Number of iterations in LSQR =  165
Number of iterations in LSQR =  59
Number of iterations in LSQR =  80
Number of iterations in LSQR =  165
Number of iterations in LSQR =  60
Number of iterations in LSQR =  80
Number of iterations in LSQR =  163
Number of iterations in LSQR =  59
Number of iterations in LSQR =  79
Number of iterations in LSQR =  164
Number of iterations in LSQR =  59
Number of iterations in LSQR =  80
Number of iterations in LSQR =  165
Number of 

Number of iterations in LSQR =  383
Number of iterations in LSQR =  480
Number of iterations in LSQR =  320
Number of iterations in LSQR =  390
Number of iterations in LSQR =  480
Number of iterations in LSQR =  322
Number of iterations in LSQR =  383
Number of iterations in LSQR =  480
Number of iterations in LSQR =  311
Number of iterations in LSQR =  373
Number of iterations in LSQR =  480
Number of iterations in LSQR =  315
Number of iterations in LSQR =  371
Number of iterations in LSQR =  480
Number of iterations in LSQR =  314
Number of iterations in LSQR =  373
Number of iterations in LSQR =  480
Number of iterations in LSQR =  313
Number of iterations in LSQR =  372
epoch =  20  loss =  1.423821991948404e-08  rmse =  9.766311424125041e-05  coeff =  1.0
norm of x  0 (0.09593389685174437+0j)
norm of x  1 (2.722484346200512+0j)
norm of x  2 (2.332047998461584+0j)
Number of iterations in LSQR =  480
Number of iterations in LSQR =  303
Number of iterations in LSQR =  381
Number of 

In [ ]:
print(se_alpha)

In [ ]:
plt.plot(alphas, se_alpha, marker='o')
plt.xlabel(r'$\alpha$')
plt.ylabel('Squared errors')
plt.xscale('log')
plt.yscale('log')
#plt.legend(loc='best')

In [ ]:
# OK, let us alpha = 0.1.
# An optimal value can be estimated by the elbow method (refer to the SpM paper by J. Otsuki et al.)
#alpha_best = 0.1
#opt_alpha_idx = 0
alpha_best = alphas[opt_alpha_idx]
print("best value of alpha", alpha_best)
#alpha_best = 0.1

coeffs = coeffs_alpha[opt_alpha_idx].reshape((3, 2, 2, Nl, Nl))
#print(coeffs.shape)

In [ ]:
# prj: [(n_sp, 3, 2, 2, Nl), (n_sp, 3, 2, 2, Nl)]
def compute_Giwn(prj, coeff):
    niw = prj[0].shape[0]
    #print(prj[0].shape)
    prj_mat = numpy.einsum('nrl,nrm->nrlm', prj[0], prj[1]).reshape((niw, 3 * 2 * 2 * Nl * Nl))
    return prj_mat@coeffs.reshape((3 * 2 * 2 * Nl * Nl))

Giwn_check = compute_Giwn(prj_check, coeffs)
print(Giwn_check.shape)
Giwn_check_ref = numpy.array([_G2_conn_ph(U, beta, n1n2[0], n1n2[1], boson_freq) for n1n2 in n1n2_check])

diff = numpy.amax(numpy.abs(Giwn_check - Giwn_check_ref))
print(diff)
print(diff < 1e-3)

In [ ]:
Giwn_reconst = compute_Giwn(prj, coeffs)
plt.plot(Giwn_reconst.real, c='r')
plt.plot(Giwn.real)
print(Giwn.real)

In [ ]:
import matplotlib.pylab as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

N = 100
N_zoom = 30
#x = numpy.linspace(-N, N, 2*N+1)
x = numpy.arange(-N, N+1)
G2_conn_ph = numpy.zeros((2 * N + 1,2 * N + 1), dtype=float)

for i in range(2 * N + 1):
    for j in range(2 * N + 1):
        n = x[i]
        np = x[j]
        G2_conn_ph[i, j] = _G2_conn_ph(U, beta, n, np, boson_freq)

vrange = numpy.amax(numpy.abs(G2_conn_ph))
plt.imshow(G2_conn_ph.real.T[N - N_zoom: N + N_zoom, N - N_zoom: N + N_zoom],
           origin="lower", extent=(-N_zoom, N_zoom, -N_zoom, N_zoom), 
           cmap=cm.bwr, vmin=-vrange/ 2, vmax=vrange / 2
          )
plt.colorbar()
print (vrange)

#plt.legend(loc='best',shadow=True,frameon=False)

In [ ]:
n1n2_check_2d = numpy.array([(x[i], x[j]) for i in range(2 * N + 1) for j in range(2 * N + 1)])
prj_check_2d = construct_prj(n1n2_check_2d)

Giwn_check = compute_Giwn(prj_check_2d, coeffs)
Giwn_check = Giwn_check.reshape((2 * N + 1, 2 * N + 1))
vrange = numpy.amax(numpy.abs(Giwn_check))
plt.imshow(Giwn_check.real.T[N - N_zoom: N + N_zoom, N - N_zoom: N + N_zoom],
           origin="lower", extent=(-N_zoom, N_zoom, -N_zoom, N_zoom), 
           cmap=cm.bwr, vmin=-vrange/ 2, vmax=vrange / 2
          )
plt.colorbar()

In [ ]:
title_size = 20
vert_range = 1.
scale_factor = 100

fig = plt.figure(figsize = (6 * fig_width, 6 * fig_height), frameon=False)

ax = fig.add_subplot(2, 2, 1)
nu_value = 0
ax.plot(scale_factor * Giwn_check.real.reshape((2*N + 1, 2*N + 1))[:, N + nu_value], 
        label="interpolation", ls='--', marker='x')
ax.plot(scale_factor * G2_conn_ph.real.T[:, N + nu_value], label="exact", lw=2)
ax.set_ylim([-7, 2])
ax.legend()
plt.title(r"$\nu=$" + str(nu_value), fontsize=title_size)

ax = fig.add_subplot(2, 2, 2)
nu_value = -10
ax.plot(scale_factor * Giwn_check.real.reshape((2*N + 1, 2*N + 1))[:, N + nu_value], 
        label="interpolation", ls='--', marker='x')
ax.plot(scale_factor * G2_conn_ph.real.T[:, N + nu_value], label="exact", lw=2)
ax.set_ylim([-7, 2])
plt.title(r"$\nu=$" + str(nu_value), fontsize=title_size)

ax = fig.add_subplot(2, 2, 3)
nu_value = -5
ax.plot(scale_factor * Giwn_check.real.reshape((2*N + 1, 2*N + 1))[:, N + nu_value], 
        label="interpolation", ls='--', marker='x')
ax.plot(scale_factor * G2_conn_ph.real.T[:, N + nu_value], label="exact", lw=2)
ax.set_ylim([-7, 2])
plt.title(r"$\nu=$" + str(nu_value), fontsize=title_size)

ax = fig.add_subplot(2, 2, 4)
nu_value = 10
ax.plot(scale_factor * Giwn_check.real.reshape((2*N + 1, 2*N + 1))[:, N + nu_value], 
        label="interpolation", ls='--', marker='x')
ax.plot(scale_factor * G2_conn_ph.real.T[:, N + nu_value], label="exact", lw=2)
ax.set_ylim([-7, 2])
plt.title(r"$\nu=$" + str(nu_value), fontsize=title_size)

plt.tight_layout()
plt.savefig("shapes_comp_wide.pdf")

In [ ]:
title_size = 20
vert_range = 1.
scale_factor = 100

fig = plt.figure(figsize = (6 * fig_width, 6 * fig_height), frameon=False)

ax = fig.add_subplot(2, 2, 1)
nu_value = 0
ax.plot(scale_factor * Giwn_check.real.reshape((2*N + 1, 2*N + 1))[N - N_zoom: N + N_zoom, N + nu_value], 
        label="interpolation", ls='--', marker='x')
ax.plot(scale_factor * G2_conn_ph.real.T[N - N_zoom: N + N_zoom, N + nu_value], label="exact", lw=2)
ax.set_ylim([-7, 2])
ax.legend()
plt.title(r"$\nu=$" + str(nu_value), fontsize=title_size)

ax = fig.add_subplot(2, 2, 2)
nu_value = -10
ax.plot(scale_factor * Giwn_check.real.reshape((2*N + 1, 2*N + 1))[N - N_zoom: N + N_zoom, N + nu_value], 
        label="interpolation", ls='--', marker='x')
ax.plot(scale_factor * G2_conn_ph.real.T[N - N_zoom: N + N_zoom, N + nu_value], label="exact", lw=2)
ax.set_ylim([-7, 2])
plt.title(r"$\nu=$" + str(nu_value), fontsize=title_size)

ax = fig.add_subplot(2, 2, 3)
nu_value = -5
ax.plot(scale_factor * Giwn_check.real.reshape((2*N + 1, 2*N + 1))[N - N_zoom: N + N_zoom, N + nu_value], 
        label="interpolation", ls='--', marker='x')
ax.plot(scale_factor * G2_conn_ph.real.T[N - N_zoom: N + N_zoom, N + nu_value], label="exact", lw=2)
ax.set_ylim([-7, 2])
plt.title(r"$\nu=$" + str(nu_value), fontsize=title_size)

ax = fig.add_subplot(2, 2, 4)
nu_value = 10
ax.plot(scale_factor * Giwn_check.real.reshape((2*N + 1, 2*N + 1))[N - N_zoom: N + N_zoom, N + nu_value], 
        label="interpolation", ls='--', marker='x')
ax.plot(scale_factor * G2_conn_ph.real.T[N - N_zoom: N + N_zoom, N + nu_value], label="exact", lw=2)
ax.set_ylim([-7, 2])
plt.title(r"$\nu=$" + str(nu_value), fontsize=title_size)

plt.tight_layout()
plt.savefig("shapes_comp_zoom.pdf")